### Treasury spot–futures basis: overview

- The basis is the difference between the futures-implied risk-free rate on the deferred Treasury futures contract and the corresponding USD OIS rate aligned to the contract's time-to-maturity.
- Positive basis means futures imply higher financing than OIS.

What follows:
- Plot of our computed basis for 2Y, 5Y, 10Y, 20Y, 30Y.
- Side-by-side overlay with Siriwardane et al.'s published series for an eyeball check.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from settings import config
from calc_treasury_data import load_treasury_sf_output
import load_bases_data

DATA_DIR = config("DATA_DIR")
OUTPUT_DIR = config("OUTPUT_DIR")

tenors = [
    "Treasury_SF_2Y",
    "Treasury_SF_5Y",
    "Treasury_SF_10Y",
    "Treasury_SF_20Y",
    "Treasury_SF_30Y",
]


In [ ]:
df_mine = load_treasury_sf_output(data_dir=DATA_DIR).sort_values("Date")
ax = df_mine.set_index("Date")[tenors].plot(figsize=(12, 6))
ax.set_title("Treasury Spot–Futures Basis (ours)")
ax.set_ylabel("Basis (bps)")
ax.set_xlabel("")
ax.grid(True)
plt.show()


In [ ]:
df_ref = load_bases_data.load_combined_spreads_wide(data_dir=DATA_DIR, raw=False, rename=True)
df_ref = df_ref.reset_index().rename(columns={"date": "Date"})
df_ref["Date"] = pd.to_datetime(df_ref["Date"]).dt.tz_localize(None)
ref_map = {
    "Treasury_SF_02Y": "Treasury_SF_2Y",
    "Treasury_SF_05Y": "Treasury_SF_5Y",
    "Treasury_SF_10Y": "Treasury_SF_10Y",
    "Treasury_SF_20Y": "Treasury_SF_20Y",
    "Treasury_SF_30Y": "Treasury_SF_30Y",
}
df_ref = df_ref.rename(columns=ref_map)
tenors_plot = [c for c in tenors if c in df_ref.columns]

fig, axes = plt.subplots(len(tenors_plot), 1, figsize=(12, 2.6 * len(tenors_plot)), sharex=True)
if len(tenors_plot) == 1:
    axes = [axes]

for i, col in enumerate(tenors_plot):
    ax = axes[i]
    df_mine.set_index("Date")[col].plot(ax=ax, label="This project", color="C0")
    df_ref.set_index("Date")[col].plot(ax=ax, label="Siriwardane et al.", color="C1", alpha=0.7)
    ax.set_title(col)
    ax.set_ylabel("bps")
    ax.grid(True)

axes[0].legend()
plt.xlabel("")
plt.tight_layout()
plt.show()


### How the Treasury spot–futures basis is computed (summary)

Inputs loaded in `calc_treasury_data.py`:
- `treasury_df.csv`: contract strings, implied repo rates, volumes, prices for near/deferred contracts.
- `ois_df.csv`: USD OIS tenors (1W, 1M, 3M, 6M, 1Y).
- `last_day_df.csv`: month/year → settlement day mapping for futures.

Computation outline:
- Parse contract strings (e.g., "DEC 21") to month/year; merge settlement day; build a maturity date and compute TTM in days for each contract.
- Interpolate OIS to the contract’s TTM using a piecewise-linear rule across 1W/1M/3M/6M/1Y.
- Define basis for the deferred contract: `(Implied_Repo_2 − OIS_2) × 100` bps.
- Clean: flag outliers via a ±45-day rolling MAD by tenor; set flagged values to NaN; require volume in the deferred contract.
- Pivot long → wide; produce series `Treasury_SF_2Y, 5Y, 10Y, 20Y, 30Y`; forward-fill small gaps.

See `calc_treasury_data.py` functions for details: `parse_contract_date`, `interpolate_ois`, `rolling_outlier_flag`, `compute_treasury_long`, `compute_treasury_output`. 



In [ ]:
# Compatibility guard for a legacy plotting cell below
import pandas as pd

df_filtered = pd.DataFrame({"Date": pd.to_datetime([])})
plot_filename = OUTPUT_DIR / "ignore_rolling_volatility.png"


Until Now


In [ ]:


# Define directories
DATA_DIR = config("DATA_DIR")

# Define file path
output_file = os.path.join(DATA_DIR, "treasury_sf_output.csv")

# Load the dataset and ensure "Date" is parsed correctly
df_output = pd.read_csv(output_file, parse_dates=["Date"])

# Define start date and dynamically get the last available date
start_date = "2011-09-27"
end_date = df_output["Date"].max()  # Automatically detect the last date in dataset

# Filter data within the date range
df_filtered = df_output[(df_output["Date"] >= start_date) & (df_output["Date"] <= end_date)].copy()

# Plot the filtered data
df_filtered.set_index("Date").plot(figsize=(12, 6))  # Set Date as index for better plotting

# Set y-axis limits
plt.ylim(-300, 200)

# Define the plot filename dynamically based on the date range
plot_filename = os.path.join(OUTPUT_DIR, f"treasury_sf_data_{start_date}_to_{end_date.date()}.png")

# Your plotting code
plt.title(f"Treasury Spot-Futures Basis \n(Positive indicates futures-implied risk-free rate is high relative to OIS)")  # Format title dynamically
plt.xlabel("Date")
plt.ylabel("Values")
plt.grid(True)

# Save the figure in OUTPUT_DIR
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

print(f"Plot has been saved successfully in {OUTPUT_DIR} as treasury_sf_data_{start_date}_to_{end_date.date()}.png!")

In [ ]:
import matplotlib.dates as mdates

# Define start date and dynamically get the last available date
start_date = "2022"
end_date = df_output["Date"].max()

# Filter data within the date range
df_filtered = df_output[(df_output["Date"] >= start_date) & (df_output["Date"] <= end_date)].copy()


In [ ]:
df_filtered.info()

In [ ]:
import matplotlib.dates as mdates

# Define start date and dynamically get the last available date
start_date = "2022"
end_date = df_output["Date"].max()

# Filter data within the date range
df_filtered = df_output[(df_output["Date"] >= start_date) & (df_output["Date"] <= end_date)].copy()

# Plot the filtered data
ax = df_filtered.set_index("Date").plot(figsize=(12, 6))

# Set y-axis limits
ax.set_ylim(-300, 200)

# # Set x-axis major ticks at every 6 months (Jan 1 and June 1)
# locator = mdates.MonthLocator(bymonth=(1, 6), bymonthday=1)
# formatter = mdates.DateFormatter('%b %Y')
# ax.xaxis.set_major_locator(locator)
# ax.xaxis.set_major_formatter(formatter)

# # Rotate x-axis labels for better readability
# plt.xticks(rotation=45)


# Define the plot filename dynamically based on the date range
plot_filename = os.path.join(OUTPUT_DIR, f"treasury_sf_data_{start_date}_to_{end_date.date()}.png")

# Your plotting code
plt.title(f"Treasury Spot-Futures Basis \n(Positive indicates futures-implied risk-free rate is high relative to OIS)")  # Format title dynamically
plt.xlabel("Date")
plt.ylabel("Values")
plt.grid(True)

# Save the figure in OUTPUT_DIR
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

print(f"Plot has been saved successfully in {OUTPUT_DIR} as treasury_sf_data_{start_date}_to_{end_date.date()}.png!")


In [ ]:
# Ensure Date is set as the index
df_filtered.set_index("Date", inplace=True)

# **Exclude non-numeric columns** before computing rolling standard deviation
df_numeric = df_filtered.select_dtypes(include=["number"])

# Compute rolling 30-day standard deviation (volatility)
rolling_volatility = df_numeric.rolling(window=30).std()

# Plot rolling volatility
plt.figure(figsize=(12, 6))
rolling_volatility.plot(ax=plt.gca())

# Define the plot filename
plot_filename = os.path.join(OUTPUT_DIR, "rolling_volatility_treasury_sf_now.png")

# Set plot title and labels
plt.title("30-Day Rolling Volatility of Treasury Spot-Futures Arbitrage Spread")
plt.xlabel("Date")
plt.ylabel("Rolling Volatility (bps)")
plt.ylim(0, 60)

# Show legend
plt.legend(title="Maturity", loc="upper right")

# Show grid
plt.grid(True)

# Save the figure in OUTPUT_DIR
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

print(f"Plot has been saved successfully in {OUTPUT_DIR} as rolling_volatility_treasury_sf_now.png!")

### What is the Treasury spot–futures basis trade?

- **Idea**: Exploit deviations between the financing rate implied by deferred Treasury futures and the USD OIS curve for the same maturity horizon.
- **Mechanics (high level)**: Construct a hedged “cash-and-carry” or “reverse cash-and-carry” using the deliverable Treasury bond (or basket) versus the futures. Direction depends on the sign of the basis and carry considerations.
- **Signal**: Our series `Treasury_SF_2Y/5Y/10Y/20Y/30Y` measure `(futures-implied rf − OIS)` in bps for the deferred contract. Persistent positive values suggest futures-implied financing is rich to OIS; negative values suggest the opposite.
- **Caveats**: Delivery options, cheapest-to-deliver dynamics, funding frictions, margining, and liquidity constraints matter; the series is an indicator, not a PnL.

